## Weather API Dashboard
This code extracts weather data for 4 countries for the last 14 years to be presented in a dashboard

In [72]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import date
import time

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# define a function that gives the lat & lon for a country
def country_coord(name):
    if name == 'Malaysia':
        return 2.5,112.5
    elif name == 'UK':
        return 54.7584,-2.6953
    elif name == 'Japan':
        return 35.6854,139.7531
    elif name == 'Spain':
        return 40,-4
    else: 
        return "Please enter a country in the list [Malaysia,UK,Japan,Spain]"
    
# list the countries we want
country_list = ['Malaysia','UK','Japan','Spain']

# date range
today = date.today()

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
df_list = []
for country in country_list:
    params = {
        "latitude": country_coord(country)[0],
        "longitude": country_coord(country)[1],
        "start_date": "2010-01-01",
        "end_date": today,
        "daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "apparent_temperature_max", "apparent_temperature_min", "apparent_temperature_mean", "sunrise", "sunset", "daylight_duration", "sunshine_duration", "precipitation_sum", "rain_sum", "precipitation_hours", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum", "et0_fao_evapotranspiration"],
        "timezone": "GMT"
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process daily data. The order of variables needs to be the same as requested.
    daily = response.Daily()
    daily_weather_code = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
    daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
    daily_temperature_2m_mean = daily.Variables(3).ValuesAsNumpy()
    daily_apparent_temperature_max = daily.Variables(4).ValuesAsNumpy()
    daily_apparent_temperature_min = daily.Variables(5).ValuesAsNumpy()
    daily_apparent_temperature_mean = daily.Variables(6).ValuesAsNumpy()
    daily_sunrise = daily.Variables(7).ValuesAsNumpy()
    daily_sunset = daily.Variables(8).ValuesAsNumpy()
    daily_daylight_duration = daily.Variables(9).ValuesAsNumpy()
    daily_sunshine_duration = daily.Variables(10).ValuesAsNumpy()
    daily_precipitation_sum = daily.Variables(11).ValuesAsNumpy()
    daily_rain_sum = daily.Variables(12).ValuesAsNumpy()
    daily_precipitation_hours = daily.Variables(13).ValuesAsNumpy()
    daily_wind_speed_10m_max = daily.Variables(14).ValuesAsNumpy()
    daily_wind_gusts_10m_max = daily.Variables(15).ValuesAsNumpy()
    daily_wind_direction_10m_dominant = daily.Variables(16).ValuesAsNumpy()
    daily_shortwave_radiation_sum = daily.Variables(17).ValuesAsNumpy()
    daily_et0_fao_evapotranspiration = daily.Variables(18).ValuesAsNumpy()

    daily_data = {"date": pd.date_range(
        start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
        end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )}
    daily_data["country"] = country
    daily_data["weather_code"] = daily_weather_code
    daily_data["temperature_2m_max"] = daily_temperature_2m_max
    daily_data["temperature_2m_min"] = daily_temperature_2m_min
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
    daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
    daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
    daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
    daily_data["sunrise"] = daily_sunrise
    daily_data["sunset"] = daily_sunset
    daily_data["daylight_duration_sec"] = daily_daylight_duration
    daily_data["sunshine_duration_sec"] = daily_sunshine_duration
    daily_data["precipitation_sum"] = daily_precipitation_sum
    daily_data["rain_sum"] = daily_rain_sum
    daily_data["precipitation_hours"] = daily_precipitation_hours
    daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
    daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
    daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
    daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
    daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration

    df = pd.DataFrame(data = daily_data)
    df_list.append(df)
    time.sleep(60)

df_final = pd.concat(df_list,ignore_index=True)
df_final.to_csv('/Users/zaineisa/Documents/VSCode/historic_weather/df.csv',mode='w')
print(len(df_final))


Coordinates 2.4956061840057373°N 112.55404663085938°E
Elevation 272.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 54.72759246826172°N -2.845855712890625°E
Elevation 71.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 35.6766242980957°N 139.80694580078125°E
Elevation 25.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 39.964847564697266°N -4.0223388671875°E
Elevation 512.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
20968


In [74]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd
from datetime import date

################################## DATA ###############################################

df = pd.read_csv('/Users/zaineisa/Documents/VSCode/historic_weather/df.csv')

app = Dash(__name__, external_stylesheets=['https://fonts.googleapis.com/css2?family=Montserrat:ital,wght@0,100..900;1,100..900&display=swap'])

styles = {
    'fontFamily': 'Montserrat',
    'textAlign':'left',
    'color':'white'
    #'color': '#1a2d3b'
}

today = date.today()
################################## LAYOUT ###############################################

app.layout = html.Div([
    html.H2(children='Historic Weather Dashboard', style=styles),
    dcc.RadioItems(df['country'].unique(), 'Malaysia', id='dropdown-selection',style=styles),
    html.Br(),
    dcc.DatePickerRange(
        id='datepicker',
        min_date_allowed=df['date'].min(),
        max_date_allowed=df['date'].max(),
        start_date=date(2014,1,1),
        end_date=today,
        display_format = 'DD MMMM YY',
        style=styles
        ),
    html.Br(),
    html.Br(),
    dcc.Graph(id='graph-content')
])

################################## CALLBACK + GRAPH ###############################################

@callback(
    Output('graph-content', 'figure'),
    [Input('dropdown-selection', 'value'),
    Input('datepicker', 'start_date'),
    Input('datepicker', 'end_date')]

)
def update_graph(value,start_date,end_date):
    dff = df[(df['date'] >= start_date) & (df['date'] <= end_date) & (df['country']==value)]
    fig = px.line(dff, x='date', y=['temperature_2m_mean','temperature_2m_max','temperature_2m_min'])
    fig.update_xaxes(
                rangeslider_visible=False,
                rangeselector=dict(
                    buttons=list([
                        dict(count=1, label="1m", step="month", stepmode="backward"),
                        dict(count=6, label="6m", step="month", stepmode="backward"),
                        #dict(count=1, label="YTD", step="year", stepmode="todate"),
                        dict(count=1, label="1y", step="year", stepmode="backward"),
                        dict(count=3, label="3y", step="year", stepmode="backward"),
                        dict(count=5, label="5y", step="year", stepmode="backward"),
                        dict(step="all")
                    ])
                )
            )
    fig.update_layout(
        xaxis_title='Timeline',
        xaxis=dict(
            showline=True,
            showgrid=True,
            showticklabels=True,
            tickfont=dict(
                family='Montserrat',
                size=12,
                color='#1a2d3b',
            ),
        ),
        yaxis_title='Temp (C)',
        yaxis=dict(
            showgrid=True,
            zeroline=False,
            showline=True,
            showticklabels=True,
            tickfont=dict(
                family='Montserrat',
                size=12,
                color='#1a2d3b',
            ),
        ),
        showlegend=True,
        legend_title='',
        plot_bgcolor='white'
    )

    fig.update_traces(name='Avg',line_color='#d4bfeb', selector=dict(name='temperature_2m_mean'))
    fig.update_traces(line_dash='dash', name='Max',line_color='#c1d0f1', selector=dict(name='temperature_2m_max'))
    fig.update_traces(line_dash='dash', name='Min',line_color='#c5f2f7', selector=dict(name='temperature_2m_min'))
    return fig

if __name__ == '__main__':
    app.run(debug=True)
